In [2]:
import os,sys,re

import time

from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn import cluster
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve,auc
from keras.callbacks import EarlyStopping

import matplotlib as mpl
import numpy as np
import pandas as pd

import sklearn
from matplotlib import pyplot as plt

In [3]:
#os.environ["CUDA_VISIBLE_DEVICES"] = '0'

import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
 
#KTF.set_session(tf.Session(config=tf.ConfigProto(device_count={'gpu':0})))


In [4]:
# os.environ["CUDA_VISIBLE_DEVICES"] = "1" #（其中0.1是选择所调用的gpu）

# gpu_options = tf.GPUOptions(allow_growth=True)

# sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))


#指定第一块GPU可用 
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
config = tf.ConfigProto() 
#不全部占满显存, 按需分配
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

KTF.set_session(sess)


In [5]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.95
session = tf.Session(config=config)

# 设置session
KTF.set_session(session )


In [6]:


from keras import backend as K

from keras.optimizers import Adam
from keras.models import *
from keras.layers import *
from keras.utils.np_utils import to_categorical

from keras.callbacks import ModelCheckpoint

In [7]:
def read_svm(file):
    encodings = []
    labels = []
    with open(file) as f:
        records = f.readlines()

    for line in records:
        line = re.sub('\d+:', '', line)
        array = line.strip().split() if line.strip() != '' else None
        encodings.append(array[1:])
        labels.append(int(array[0]))

    return np.array(encodings).astype(float), np.array(labels).astype(int)

In [8]:
def draw_ROC_curve(y_test,y_predict,savepath):
    '''
    画ROC曲线
    '''
    false_positive_rate,true_positive_rate,thresholds=roc_curve(y_test, y_predict)
    roc_auc=auc(false_positive_rate, true_positive_rate)
    plt.title('ROC')
    plt.plot(false_positive_rate, true_positive_rate,'b',label='AUC = %0.2f'% roc_auc)
    plt.legend(loc='lower right')
    plt.plot([0,1],[0,1],'r--')
    plt.ylabel('TPR')
    plt.xlabel('FPR')
    plt.savefig(savepath)
    plt.close(0)

In [9]:
# AUC for a binary classifier
def auc_1(y_true, y_pred):
    ptas = tf.stack([binary_PTA(y_true,y_pred,k) for k in np.linspace(0, 1, 1000)],axis=0)
    pfas = tf.stack([binary_PFA(y_true,y_pred,k) for k in np.linspace(0, 1, 1000)],axis=0)
    pfas = tf.concat([tf.ones((1,)) ,pfas],axis=0)
    binSizes = -(pfas[1:]-pfas[:-1])
    s = ptas*binSizes
    return K.sum(s, axis=0)

# PFA, prob false alert for binary classifier
def binary_PFA(y_true, y_pred, threshold=K.variable(value=0.5)):
    y_pred = K.cast(y_pred >= threshold, 'float32')
    # N = total number of negative labels
    N = K.sum(1 - y_true)
    # FP = total number of false alerts, alerts from the negative class labels
    FP = K.sum(y_pred - y_pred * y_true)
    return FP/N

# P_TA prob true alerts for binary classifier
def binary_PTA(y_true, y_pred, threshold=K.variable(value=0.5)):
    y_pred = K.cast(y_pred >= threshold, 'float32')
    # P = total number of positive labels
    P = K.sum(y_true)
    # TP = total number of correct alerts, alerts from the positive class labels
    TP = K.sum(y_pred * y_true)
    return TP/P
 


In [10]:
# 分割并序列编码
AA = '-GAVLIFWYDNEKQMSTCPHR'
def pep(path, seq_len):
    seqs = open(path).readlines()
    cut = (len(seqs[0].split()[0]) - 1 - seq_len) // 2
    X = [[AA.index(res.upper()) if res.upper() in AA else 0
          for res in (seq.split()[0][cut:-cut] if cut != 0 else seq.split()[0])]
        for seq in seqs if seq.strip() != '']
    y = [int(seq.split()[-1]) for seq in seqs if seq.strip() != '']
    return np.array(X), np.array(y)


In [ ]:
path1 = 'C:/Users/Crow/Desktop/human_data/Step_11_CV/Train.txt'
path2 = 'C:/Users/Crow/Desktop/human_data/Step_11_IND/Independent.txt'
path_train =  'C:/Users/Crow/Desktop/human_data/Step_11_CV/Train_29_EGAAC_cv.txt'
path_train2 = 'C;/Users/Crow/Desktop/human_data/Step_11_CV/Train_29_EGAAC_gap4.txt'
path_test =  'C:/Users/Crow/Desktop/human_data/Step_11_IND/Test_29_EGAAC_gap4.txt'


plant_train = 'C:/Users/Crow/Desktop/plant_data/Step_11_CV/Train.txt'
plant_test = 'C:/Users/Crow/Desktop/plant_data/Step_11_IND/Independent.txt'

In [ ]:
x_plant_train,y_plant_train = pep(plant_train,25)
x_plant_test,y_plant_test = pep(plant_test,25)

In [ ]:
train = read_svm(path_train)
test = read_svm(path_test)

x_train = train[0]
y_train = train[1]

x_test = test[0]
y_test = test[1]



In [ ]:
x_train2,y_train2 = pep(path1,29)
x_test2,y_test2 = pep(path2,29)

In [ ]:
#RNN(embed_input_dim, embed_output_dim, lstm_dim, input_length, dropout=0.2, weights=None)
model = Sequential()
model.add(Embedding(embed_input_dim, embed_output_dim, input_length=input_length, weights=weights, trainable=True))

# model.add(GRU(lstm_dim, implementation=2, return_sequences=True))
# model.add(Dropout(dropout))

model.add(GRU(lstm_dim, implementation=2))
model.add(Dropout(dropout))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(dropout))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# 全连接网络
model = Sequential()
model.add(Dense(130, input_shape=(x_train.shape[1],)))
model.add(Activation('tanh'))
model.add(Dropout(0.3))
model.add(Dense(130))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(130))
model.add(Activation('tanh'))
model.add(Dropout(0.3))
model.add(Dense(130))
model.add(Activation('linear'))
model.add(Dense(1)) # 这里需要和输出的维度一致
model.add(Activation('sigmoid'))

# For a multi-class classification problem
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=4)
callbacks_list = [early_stopping]

epochs = 100
model.fit(x_train, y_train, epochs=epochs, batch_size=260, shuffle=True,validation_split = 0.2,
         callbacks=callbacks_list)

pre = model.predict_proba(x_test)
fpr, tpr, thresholds = roc_curve(y_test,pre,pos_label=1)
print(sklearn.metrics.auc(fpr, tpr))

In [ ]:
# 一维卷积测试




model = Sequential()
model.add(Conv1D(130, 9, strides=1, padding='valid', dilation_rate=1, activation=None, 
                          use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', 
                          kernel_regularizer=None, bias_regularizer=None, 
                          activity_regularizer=None, kernel_constraint=None, bias_constraint=None, input_shape=(x_train.shape[0],130)))

model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=200, epochs=10)

In [ ]:
#使用LSTM的序列分类


model = Sequential()
#model.add(Embedding(130, output_dim=256))
model.add(Embedding(22, 64, input_length = 31))

model.add(LSTM(256))

model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


# filepath="C:/Users/Crow/Desktop/human_data/LSTM/checkpoint-{val_loss:.2f}-{epoch:02d}e-val_acc_{val_acc:.2f}.hdf5"

# checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True,mode='max')
early_stopping = EarlyStopping(monitor='val_loss', patience=4)
callbacks_list = [early_stopping]

model.fit(x_train2, y_train2, batch_size=1024, epochs=25, shuffle=True,validation_split = 0.2,
         callbacks=callbacks_list, verbose=1)

#score = model.evaluate(x_test, y_test, batch_size=16)
pre = model.predict_proba(x_test2)
fpr, tpr, thresholds = roc_curve(y_test2,pre,pos_label=1)
print(model.evaluate(x_test2,y_test2))
roc_auc = sklearn.metrics.auc(fpr, tpr)
print(sklearn.metrics.auc(fpr, tpr))
draw_ROC_curve(y_test2,pre,savepath='C:/Users/Crow/Desktop/human_data/LSTM_12.5.png')

In [ ]:
roc_auc = sklearn.metrics.auc(fpr, tpr)
print(sklearn.metrics.auc(fpr, tpr))
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.3f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
for line in locals()['In']:
    print(line)

In [ ]:
model = Sequential()
model.add(Dense(32, input_shape=(x_train.shape[1],)))
model.add(Activation('tanh'))
model.add(Dropout(0.5))
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(32))
model.add(Activation('linear'))
model.add(Dense(1)) # 这里需要和输出的维度一致
model.add(Activation('sigmoid'))

# For a multi-class classification problem
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=[auc_1])
epochs = 100
model.fit(x_train, y_train, epochs=epochs, batch_size=256, validation_split=0.1, shuffle=True)

pre = model.predict_proba(x_test)
fpr, tpr, thresholds = roc_curve(y_test,pre,pos_label=1)
print(sklearn.metrics.auc(fpr, tpr))

In [ ]:
pre = model.predict_proba(x_test)
fpr, tpr, thresholds = roc_curve(y_test,pre,pos_label=1)
print(sklearn.metrics.auc(fpr, tpr))

In [ ]:

neurons=7
window=120
dropout=0.3
model = Sequential()
model.add(Embedding(1000, 32, input_length = 31))

model.add(Conv1D(neurons, window, activation='relu', padding='same'))
model.add(MaxPooling1D(2))
model.add(Dropout(dropout))
model.add(Conv1D(neurons, window, activation='relu', padding='same'))
model.add(MaxPooling1D(2))
model.add(Dropout(dropout))
model.add(Conv1D(neurons, window, activation='relu', padding='same'))
model.add(MaxPooling1D(2))
model.add(Dropout(dropout))
model.add(Conv1D(neurons, window, activation='relu', padding='same'))
model.add(MaxPooling1D(2))
model.add(Dropout(dropout))


model.add(Flatten())


model.add(Dense(neurons, activation='relu'))
model.add(Dropout(dropout))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])




filepath="C:/Users/Crow/Desktop/human_data/CNN/checkpoint-{val_loss:.2f}-{epoch:02d}e-val_acc_{val_acc:.2f}.hdf5"

checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True,mode='max', period=4)
early_stopping = EarlyStopping(monitor='val_loss', patience=4)
callbacks_list = [checkpoint,early_stopping]


# epochs = 9
# history = model.fit(x_train2, y_train2, epochs=epochs, batch_size=256, validation_split=0.1, shuffle=True,callbacks=callbacks_list)
# pre = model.predict(x_test2)
# fpr, tpr, thresholds = roc_curve(y_test2,pre,pos_label=1)
# print(sklearn.metrics.auc(fpr, tpr))
# draw_ROC_curve(y_test2,pre,savepath='C:/Users/Crow/Desktop/human_data/CNN_11.28.png')






model.fit(x_train2, y_train2,
                   epochs = 20,
                   batch_size = 256,
                   shuffle=True,validation_split = 0.2,
         callbacks=callbacks_list, verbose=1)
print(model.evaluate(x_test2, y_test2, batch_size=256))
pre = model.predict(x_test2)
fpr, tpr, thresholds = roc_curve(y_test2,pre,pos_label=1)
print(sklearn.metrics.auc(fpr, tpr))
draw_ROC_curve(y_test2,pre,savepath='C:/Users/Crow/Desktop/human_data/CNN_11.30.png')


In [ ]:
pre2 = model.predict(x_test2)
fpr, tpr, thresholds = roc_curve(y_test2,pre2,pos_label=1)
print(sklearn.metrics.auc(fpr, tpr))

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
model = Sequential()
model.add(Conv2D(32,(5,5),strides=(1,1),input_shape=(28,28,1),padding='valid',activation='relu',kernel_initializer='uniform'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64,(5,5),strides=(1,1),padding='valid',activation='relu',kernel_initializer='uniform'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(100,activation='relu'))
model.add(Dense(10,activation='softmax'))


model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=[auc_1])
epochs = 200
model.fit(x_train, y_train, epochs=epochs, batch_size=1000, validation_split=0.1, shuffle=True)

pre = model.predict_proba(x_test)
fpr, tpr, thresholds = roc_curve(y_test,pre,pos_label=1)
print(sklearn.metrics.auc(fpr, tpr))

In [ ]:
import os
from tensorflow.python.client import device_lib
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "99"

if __name__ == "__main__":
    print(device_lib.list_local_devices())

In [ ]:
# 一维卷积测试




model = Sequential()
model.add(Embedding(x_train2.shape[0], 64, input_length = 31))

model.add(Conv1D(130, 9, strides=1, padding='valid', dilation_rate=1, activation=None, 
                          use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', 
                          kernel_regularizer=None, bias_regularizer=None, 
                          activity_regularizer=None, kernel_constraint=None, bias_constraint=None, input_shape=(x_train.shape[0],31)))

model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[auc_1])

model.fit(x_train2, y_train2, batch_size=200, epochs=10)






In [ ]:
window  = 7
neurons = 120
dropout= 0.3

model = Sequential()
model.add(Embedding(1000, 64, input_length = 31))

model.add(Conv1D(neurons, window, activation='relu', padding='same'))
model.add(MaxPooling1D(2))
model.add(Dropout(dropout))
model.add(Conv1D(neurons, window, activation='relu', padding='same'))
model.add(MaxPooling1D(2))
model.add(Dropout(dropout))
model.add(Conv1D(neurons, window, activation='relu', padding='same'))
model.add(MaxPooling1D(2))
model.add(Dropout(dropout))
model.add(Conv1D(neurons, window, activation='relu', padding='same'))
model.add(MaxPooling1D(2))
model.add(Dropout(dropout))
model.add(Flatten())
model.add(Dense(neurons, activation='relu'))
model.add(Dropout(dropout))
model.add(Dense(1, activation='sigmoid'))
  
    
history = model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

filepath="C:/Users/Crow/Desktop/human_data/CNN/checkpoint-{epoch:02d}e-val_acc_{val_acc:.2f}.hdf5"

checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True,mode='max', period=4)
early_stopping = EarlyStopping(monitor='val_loss', patience=4)
callbacks_list = [checkpoint,early_stopping]

model.fit(x_train2, y_train2,
                   epochs = 20,
                   batch_size = 1024,
                   shuffle=True,validation_split = 0.2,
         callbacks=callbacks_list, verbose=1)
print(model.evaluate(x_test2, y_test2, batch_size=256))
pre = model.predict(x_test2)
fpr, tpr, thresholds = roc_curve(y_test2,pre,pos_label=1)
print(sklearn.metrics.auc(fpr, tpr))
draw_ROC_curve(y_test2,pre,savepath='C:/Users/Crow/Desktop/human_data/CNN_11.30.png')

In [ ]:
model.save("C:/Users/Crow/Desktop/human_data/human_CNN_w7_n_120_d_0.3_l_31_0.866.hdf5")

In [ ]:
model2 = load_model('C:/Users/Crow/Desktop/human_data/human_CNN_w7_n_120_d_0.2_l_27.hdf5')

In [ ]:
pre = model2.predict(x_test2)
fpr, tpr, thresholds = roc_curve(y_test2,pre,pos_label=1)
print(sklearn.metrics.auc(fpr, tpr))


In [ ]:

# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# plant

x_plant_train,y_plant_train = pep(plant_train,29)
x_plant_test,y_plant_test = pep(plant_test,29)


window  = 7
neurons = 100
dropout= 0.3

model = Sequential()
model.add(Embedding(1000, 64, input_length = 31))

model.add(Conv1D(neurons, window, activation='relu', padding='same'))
model.add(MaxPooling1D(2))
model.add(Dropout(dropout))
model.add(Conv1D(neurons, window, activation='relu', padding='same'))
model.add(MaxPooling1D(2))
model.add(Dropout(dropout))
model.add(Conv1D(neurons, window, activation='relu', padding='same'))
model.add(MaxPooling1D(2))
model.add(Dropout(dropout))
model.add(Conv1D(neurons, window, activation='relu', padding='same'))
model.add(MaxPooling1D(2))
model.add(Dropout(dropout))
model.add(Flatten())
model.add(Dense(neurons, activation='relu'))
model.add(Dropout(dropout))
model.add(Dense(1, activation='sigmoid'))
  

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

filepath="C:/Users/Crow/Desktop/plant_data/CNN/checkpoint-{epoch:02d}e-val_acc_{val_acc:.2f}.hdf5"

checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True,mode='max', period=4)
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
callbacks_list = [checkpoint,early_stopping]

history = model.fit(x_plant_train, y_plant_train,
                   epochs = 15,
                   batch_size = 500,
                   validation_split = 0.2,shuffle=True,
         callbacks=callbacks_list, verbose=1)

print(model.evaluate(x_plant_test, y_plant_test, batch_size=256))

pre = model.predict(x_plant_test)
fpr, tpr, thresholds = roc_curve(y_plant_test,pre,pos_label=1)
print(sklearn.metrics. auc(fpr, tpr))


draw_ROC_curve(y_plant_test,pre,savepath='C:/Users/Crow/Desktop/plant_data/plant_CNN_12.4.png')




In [ ]:
model.save("C:/Users/Crow/Desktop/plant_data/plant_CNN_w7_n_100_d_0.3_l_29_0.83.hdf5")

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# plant
#使用LSTM的序列分类
x_plant_train,y_plant_train = pep(plant_train,23)
x_plant_test,y_plant_test = pep(plant_test,23)

model = Sequential()
#model.add(Embedding(130, output_dim=256))
model.add(Embedding(1000, 32, input_length = 25))

model.add(LSTM(256))

model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# model.compile(loss='binary_crossentropy',
#               optimizer='adam',
#               metrics=['accuracy',auc_1])


model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])




filepath="C:/Users/Crow/Desktop/plant_data/LSTM/checkpoint-{epoch:02d}e-val_acc_{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True,mode='max', period=4)
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
callbacks_list = [checkpoint]

history = model.fit(x_plant_train, y_plant_train,
                   epochs = 35,
                   batch_size = 256,
                   validation_split = 0.2,shuffle=True,
         callbacks=callbacks_list, verbose=1)
#score = model.evaluate(x_test, y_test, batch_size=16)
pre = model.predict_proba(x_plant_test)
fpr, tpr, thresholds = roc_curve(y_plant_test,pre,pos_label=1)
print(model.evaluate(x_plant_test,y_plant_test))
roc_auc = sklearn.metrics.auc(fpr, tpr)
print(sklearn.metrics.auc(fpr, tpr))
draw_ROC_curve(y_plant_test,pre,savepath='C:/Users/Crow/Desktop/plant_data/LSTM_plant_11_28_2.png')

In [ ]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['auc_1'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['auc_1'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


In [ ]:
def create_human_cnn():
    neurons = 120
    dropout= 0.3
    # create model
    model = Sequential()
    model.add(Embedding(1000, 64, input_length = 31))

    model.add(Conv1D(neurons, window, activation='relu', padding='same'))
    model.add(MaxPooling1D(2))
    model.add(Dropout(dropout))
    model.add(Conv1D(neurons, window, activation='relu', padding='same'))
    model.add(MaxPooling1D(2))
    model.add(Dropout(dropout))
    model.add(Conv1D(neurons, window, activation='relu', padding='same'))
    model.add(MaxPooling1D(2))
    model.add(Dropout(dropout))
    model.add(Conv1D(neurons, window, activation='relu', padding='same'))
    model.add(MaxPooling1D(2))
    model.add(Dropout(dropout))
    model.add(Flatten())
    model.add(Dense(neurons, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    #model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
    return model




In [ ]:
neurons = 120
dropout= 0.3
filepath="C:/Users/Crow/Desktop/human_data/CNN/checkpoint-{epoch:02d}e-val_acc_{val_acc:.2f}.hdf5"

checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True,mode='max', period=4)
early_stopping = EarlyStopping(monitor='val_loss', patience=4)
callbacks_list = [checkpoint,early_stopping]

model.fit(x_train2, y_train2,
                   epochs = 20,
                   batch_size = 1024,
                   shuffle=True,validation_split = 0.2,
         callbacks=callbacks_list, verbose=1)

In [ ]:
# k-fold 

kf = KFold(n_splits = 10)

for train_index, test_index in kf.split(x_train2):

    x_train3, x_test3 = x_train2[train_index], x_train2[test_index]
    y_train3, y_test3 = y_train2[train_index], y_train2[test_index]
    
    custom_resnet = create_human_cnn()
    #filepath="C:/Users/Crow/Desktop/human_data/CNN/checkpoint-{epoch:02d}e-val_acc_{val_acc:.2f}.hdf5"

    #checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True,mode='max', period=4)
    early_stopping = EarlyStopping(monitor='val_loss', patience=4)
    callbacks_list = [early_stopping]

    custom_resnet.fit(x_train3, y_train3, validation_data = (x_test3, y_test3), epochs = 20, batch_size = 1024,shuffle=True,
         callbacks=callbacks_list, verbose=1)

In [ ]:
print(custom_resnet.evaluate(x_test2, y_test2, batch_size=256))
pre = custom_resnet.predict(x_test2)
fpr, tpr, thresholds = roc_curve(y_test2,pre,pos_label=1)
print(sklearn.metrics.auc(fpr, tpr))
draw_ROC_curve(y_test2,pre,savepath='C:/Users/Crow/Desktop/human_data/CNN_k-fold_12.4.png')

In [ ]:
kf = KFold(n_splits = 10)

for train_index, test_index in kf.split(x_plant_train):

    x_train3, x_test3 = x_plant_train[train_index], x_plant_train[test_index]
    y_train3, y_test3 = y_plant_train[train_index], y_plant_train[test_index]
    
    custom_resnet = create_human_cnn()
    #filepath="C:/Users/Crow/Desktop/human_data/CNN/checkpoint-{epoch:02d}e-val_acc_{val_acc:.2f}.hdf5"

    #checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True,mode='max', period=4)
    early_stopping = EarlyStopping(monitor='val_loss', patience=4)
    callbacks_list = [early_stopping]

    custom_resnet.fit(x_train3, y_train3, validation_data = (x_test3, y_test3), epochs = 20, batch_size = 256,shuffle=True,
         callbacks=callbacks_list, verbose=1)

In [ ]:
print(custom_resnet.evaluate(x_plant_test, y_plant_test, batch_size=256))
pre = custom_resnet.predict(x_plant_test)
fpr, tpr, thresholds = roc_curve(y_plant_test,pre,pos_label=1)
print(sklearn.metrics.auc(fpr, tpr))
draw_ROC_curve(y_plant_test,pre,savepath='C:/Users/Crow/Desktop/plant_data/CNN_10-fold.png')

In [ ]:
# 手写CRF

class CRF(Layer):
    """纯Keras实现CRF层
    CRF层本质上是一个带训练参数的loss计算层，因此CRF层只用来训练模型，
    而预测则需要另外建立模型。
    """
    def __init__(self, ignore_last_label=False, **kwargs):
        """ignore_last_label：定义要不要忽略最后一个标签，起到mask的效果
        """
        self.ignore_last_label = 1 if ignore_last_label else 0
        super(CRF, self).__init__(**kwargs)
    def build(self, input_shape):
        self.num_labels = input_shape[-1] - self.ignore_last_label
        self.trans = self.add_weight(name='crf_trans',
                                     shape=(self.num_labels, self.num_labels),
                                     initializer='glorot_uniform',
                                     trainable=True)
    def log_norm_step(self, inputs, states):
        """递归计算归一化因子
        要点：1、递归计算；2、用logsumexp避免溢出。
        技巧：通过expand_dims来对齐张量。
        """
        states = K.expand_dims(states[0], 2) # (batch_size, output_dim, 1)
        trans = K.expand_dims(self.trans, 0) # (1, output_dim, output_dim)
        output = K.logsumexp(states+trans, 1) # (batch_size, output_dim)
        return output+inputs, [output+inputs]
    def path_score(self, inputs, labels):
        """计算目标路径的相对概率（还没有归一化）
        要点：逐标签得分，加上转移概率得分。
        技巧：用“预测”点乘“目标”的方法抽取出目标路径的得分。
        """
        point_score = K.sum(K.sum(inputs*labels, 2), 1, keepdims=True) # 逐标签得分
        labels1 = K.expand_dims(labels[:, :-1], 3)
        labels2 = K.expand_dims(labels[:, 1:], 2)
        labels = labels1 * labels2 # 两个错位labels，负责从转移矩阵中抽取目标转移得分
        trans = K.expand_dims(K.expand_dims(self.trans, 0), 0)
        trans_score = K.sum(K.sum(trans*labels, [2,3]), 1, keepdims=True)
        return point_score+trans_score # 两部分得分之和
    def call(self, inputs): # CRF本身不改变输出，它只是一个loss
        return inputs
    def loss(self, y_true, y_pred): # 目标y_pred需要是one hot形式
        mask = 1-y_true[:,1:,-1] if self.ignore_last_label else None
        y_true,y_pred = y_true[:,:,:self.num_labels],y_pred[:,:,:self.num_labels]
        init_states = [y_pred[:,0]] # 初始状态
        log_norm,_,_ = K.rnn(self.log_norm_step, y_pred[:,1:], init_states, mask=mask) # 计算Z向量（对数）
        log_norm = K.logsumexp(log_norm, 1, keepdims=True) # 计算Z（对数）
        path_score = self.path_score(y_pred, y_true) # 计算分子（对数）
        return log_norm - path_score # 即log(分子/分母)
    def accuracy(self, y_true, y_pred): # 训练过程中显示逐帧准确率的函数，排除了mask的影响
        mask = 1-y_true[:,:,-1] if self.ignore_last_label else None
        y_true,y_pred = y_true[:,:,:self.num_labels],y_pred[:,:,:self.num_labels]
        isequal = K.equal(K.argmax(y_true, 2), K.argmax(y_pred, 2))
        isequal = K.cast(isequal, 'float32')
        if mask == None:
            return K.mean(isequal)
        else:
            return K.sum(isequal*mask) / K.sum(mask)

In [ ]:
# keras_contrib Bi-LSTM+CRF 

from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import Dropout
from keras_contrib.layers.crf import CRF
from keras_contrib.utils import save_load_utils


VOCAB_SIZE = 2500
EMBEDDING_OUT_DIM = 128
TIME_STAMPS = 31
HIDDEN_UNITS = 200
DROPOUT_RATE = 0.3
NUM_CLASS = 5


def build_embedding_bilstm2_crf_model():
    """
    带embedding的双向LSTM + crf
    """
    model = Sequential()
    model.add(Embedding(VOCAB_SIZE, output_dim=EMBEDDING_OUT_DIM, input_length=TIME_STAMPS))
    model.add(Bidirectional(LSTM(HIDDEN_UNITS, return_sequences=True)))
    model.add(Dropout(DROPOUT_RATE))
    model.add(Bidirectional(LSTM(HIDDEN_UNITS, return_sequences=True)))
    model.add(Dropout(DROPOUT_RATE))
    model.add(TimeDistributed(Dense(NUM_CLASS)))
    crf_layer = CRF(NUM_CLASS)
    model.add(crf_layer)
    model.compile('rmsprop', loss=crf_layer.loss_function, metrics=[crf_layer.accuracy])
    return model

def save_embedding_bilstm2_crf_model(model, filename):
    save_load_utils.save_all_weights(model,filename)

def load_embedding_bilstm2_crf_model(filename):
    model = build_embedding_bilstm2_crf_model()
    save_load_utils.load_all_weights(model, filename)
    return model






model = build_embedding_bilstm2_crf_model()
model.fit(x_train2, y_train2, batch_size=256, epochs=25, callbacks=callbacks_list, verbose=1)

#score = model.evaluate(x_test, y_test, batch_size=16)
pre = model.predict_proba(x_test2)
fpr, tpr, thresholds = roc_curve(y_test2,pre,pos_label=1)
print(model.evaluate(x_test2,y_test2))
roc_auc = sklearn.metrics.auc(fpr, tpr)
print(sklearn.metrics.auc(fpr, tpr))

In [ ]:
# test

from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF

input = Input(shape=(31,))
model = Embedding(input_dim=22, output_dim=20,
                  input_length=31, mask_zero=True)(input)  # 20-dim embedding
model = Bidirectional(LSTM(units=50, return_sequences=True,
                           recurrent_dropout=0.1))(model)  # variational biLSTM
model = TimeDistributed(Dense(50, activation="relu"))(model)  # a dense layer as suggested by neuralNer
crf = CRF(2)  # CRF layer
out = crf(model)  # output

In [ ]:
model = Model(input, out)

In [ ]:
model.compile(optimizer="rmsprop", loss=crf.loss_function, metrics=[crf.accuracy])

In [ ]:
model.summary()

In [ ]:
history = model.fit(x_test2, y_test2, batch_size=32, epochs=5, validation_split=0.1, verbose=1)

In [ ]:
#test2

EMBEDDING_SIZE = 128

model = Sequential()
model.add(Embedding(23, EMBEDDING_SIZE,input_length=31))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1))
model.add(Activation("sigmoid"))
model.compile(loss="binary_crossentropy", optimizer="adam",metrics=["accuracy"])

early_stopping = EarlyStopping(monitor='val_loss', patience=4)
callbacks_list = [early_stopping]

history = model.fit(x_test2, y_test2, batch_size=32, epochs=25, validation_split=0.2, shuffle=True,
         callbacks=callbacks_list, verbose=1)


pre = model.predict_proba(x_test2)
fpr, tpr, thresholds = roc_curve(y_test2,pre,pos_label=1)
print(model.evaluate(x_test2,y_test2))
roc_auc = sklearn.metrics.auc(fpr, tpr)
print(sklearn.metrics.auc(fpr, tpr))

In [11]:
path_train = 'C:/Users/Crow/Desktop/human_data_12.12/Train.txt'
path_test =  'C:/Users/Crow/Desktop/human_data_12.12/Independent.txt'

x_train,y_train = pep(path_train,29)
x_test,y_test = pep(path_test,29)

In [33]:
x_train.shape

(73793, 29)

In [19]:
# CNN test

neurons=128
window=5
dropout=0.4
model = Sequential()
model.add(Embedding(21, 5, input_length = 31))

model.add(Conv1D(neurons, window, activation='relu', padding='same'))
model.add(MaxPooling1D(2))
model.add(Dropout(dropout))

model.add(Conv1D(neurons, window, activation='relu', padding='same'))
model.add(MaxPooling1D(2))
model.add(Dropout(dropout))

model.add(Conv1D(neurons, window, activation='relu', padding='same'))
model.add(MaxPooling1D(2))
model.add(Dropout(dropout))

model.add(Conv1D(neurons, window, activation='relu', padding='same'))
model.add(MaxPooling1D(2))
model.add(Dropout(dropout))


model.add(Flatten())


# model.add(Dense(neurons, activation='relu'))
# model.add(Dropout(dropout))

model.add(Dense(1, activation='sigmoid'))
#model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])




# filepath="C:/Users/Crow/Desktop/human_data/CNN/checkpoint-{val_loss:.2f}-{epoch:02d}e-val_acc_{val_acc:.2f}.hdf5"

# checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True,mode='max', period=4)
early_stopping = EarlyStopping(monitor='val_loss', patience=4)
callbacks_list = [early_stopping]






model.fit(x_train, y_train,
                   epochs = 20,
                   batch_size = 128,
                   shuffle=True,validation_split = 0.2,
         callbacks=callbacks_list, verbose=1)
print(model.evaluate(x_test, y_test, batch_size=256))
pre = model.predict(x_test)
fpr, tpr, thresholds = roc_curve(y_test,pre,pos_label=1)
print(sklearn.metrics.auc(fpr, tpr))
#draw_ROC_curve(y_test2,pre,savepath='C:/Users/Crow/Desktop/human_data/CNN_12.5.png')



Train on 59034 samples, validate on 14759 samples
Epoch 1/20
59034/59034 [==============================] - 4s 65us/step - loss: 0.3523 - acc: 0.8865 - val_loss: 0.2307 - val_acc: 1.0000
Epoch 2/20
59034/59034 [==============================] - 3s 54us/step - loss: 0.3154 - acc: 0.8867 - val_loss: 0.1178 - val_acc: 1.0000
Epoch 3/20
59034/59034 [==============================] - 3s 54us/step - loss: 0.2913 - acc: 0.8867 - val_loss: 0.1543 - val_acc: 1.0000
Epoch 4/20
59034/59034 [==============================] - 3s 54us/step - loss: 0.2835 - acc: 0.8867 - val_loss: 0.1468 - val_acc: 1.0000
Epoch 5/20
59034/59034 [==============================] - 3s 54us/step - loss: 0.2769 - acc: 0.8867 - val_loss: 0.1015 - val_acc: 1.0000
Epoch 6/20
59034/59034 [==============================] - 3s 54us/step - loss: 0.2699 - acc: 0.8867 - val_loss: 0.1192 - val_acc: 0.9997
Epoch 7/20
59034/59034 [==============================] - 3s 54us/step - loss: 0.2697 - acc: 0.8869 - val_loss: 0.1225 - val_acc

In [16]:
from keras.utils.vis_utils import plot_model
import os
os.environ["PATH"] += os.pathsep + 'D:/Graphviz2.38/bin'

In [20]:
plot_model(model, to_file='keras_test_model1.png',show_shapes=True)

In [23]:
# 读取model h5文件
model = load_model('C:\Users\Crow\Desktop\human_data\CNN\128_8_0.4_3conv1d_12.5.hdf5')

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (<ipython-input-23-1aa69cc6b2f5>, line 2)

In [24]:

model = load_model('c:/Users/Crow/Desktop/human_data/CNN/128_8_0.4_3conv1d_12.5.hdf5')

In [28]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_183 (Embedding)    (None, 29, 32)            704       
_________________________________________________________________
conv1d_559 (Conv1D)          (None, 29, 128)           32896     
_________________________________________________________________
max_pooling1d_559 (MaxPoolin (None, 14, 128)           0         
_________________________________________________________________
dropout_731 (Dropout)        (None, 14, 128)           0         
_________________________________________________________________
conv1d_560 (Conv1D)          (None, 14, 128)           131200    
_________________________________________________________________
max_pooling1d_560 (MaxPoolin (None, 7, 128)            0         
_________________________________________________________________
dropout_732 (Dropout)        (None, 7, 128)            0         
__________

In [29]:
x_train,y_train = pep(path_train,27)
x_test,y_test = pep(path_test,27)
print(model.evaluate(x_test, y_test, batch_size=256))
pre = model.predict(x_test)
fpr, tpr, thresholds = roc_curve(y_test,pre,pos_label=1)
print(sklearn.metrics.auc(fpr, tpr))

17980/17980 [==============================] - 0s 10us/step
[0.24421920618570686, 0.9059510567429598]
0.8653889378771376
